<a href="https://colab.research.google.com/github/msh2481/CodeStyler/blob/main/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!rm -rf ./*
!git clone https://github.com/msh2481/CodeStyler.git && mv CodeStyler/* . && rm -rf CodeStyler
!ls

Cloning into 'CodeStyler'...
remote: Enumerating objects: 7921, done.
remote: Counting objects: 100% (7921/7921), done.
remote: Compressing objects: 100% (6521/6521), done.
remote: Total 7921 (delta 1399), reused 7918 (delta 1399), pack-reused 0
Receiving objects: 100% (7921/7921), 9.07 MiB | 16.27 MiB/s, done.
Resolving deltas: 100% (1399/1399), done.
filenames.txt  files  README.md


In [68]:
from random import shuffle
from collections import deque, defaultdict, Counter
import numpy as np
import torch
import torch.nn as nn
from nn import functional as F

In [62]:
TARGET_TEXT_SIZE = 2 ** 8
ALPHABET_SIZE = 256

In [64]:
rawTexts = []
for filename in open('filenames.txt'):
    text = open(filename.strip()).read()
    parts = len(text) // TARGET_TEXT_SIZE + 1
    partLen = len(text) // parts
    for pos in range(0, len(text), partLen):
        rawTexts.append(text[pos : pos + partLen])
alphabet = sorted(list(set([ch for text in rawTexts for ch in text])))
assert len(alphabet) < ALPHABET_SIZE

In [65]:
print(f'alphabet of length {len(alphabet)}: {alphabet}')
shuffle(rawTexts)
print(f'{len(rawTexts)} texts in total')
print(rawTexts[:10])

alphabet of length 244: ['\x00', '\x02', '\t', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\xa0', '§', '©', '«', '®', '±', '²', 'µ', '·', '»', '¿', 'Ä', 'Å', 'Ö', 'Ü', 'ß', 'ä', 'å', 'ö', 'ü', 'İ', 'ı', 'ŉ', 'ſ', 'ƻ', 'Ǆ', 'ǅ', 'ǆ', 'Ǉ', 'ǈ', 'ǉ', 'Ǌ', 'ǋ', 'ǌ', 'Ǳ', 'ǲ', 'ǳ', 'ʰ', 'ʼ', 'ͅ', 'Β', 'Ε', 'Θ', 'Ι', 'Κ', 'Μ', 'Π', 'Ρ', 'Σ', 'Φ', 'Ω', 'β', 'ε', 'θ', 'ι', 'κ', 'μ', 'π', 'ρ', 'ς', 'σ', 'φ', 'ω', 'ϐ', 'ϑ', 'ϕ', 'ϖ', 'ϰ', 'ϱ', 'ϴ', 'ϵ', 'К', 'П', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф'

In [71]:
charToIndexMap = { c : i for i, c in enumerate(alphabet) }
def charToIndex(c):
    return charToIndexMap.get(c, ALPHABET_SIZE - 1)

dataset = [np.array(list(map(charToIndex, text)), dtype=np.uint8) for text in rawTexts]

In [74]:
print(dataset[0])

(254,)
[69 81 73 88 73 86 87  4 15 33  4 71 86 73 69 88 73 55 93 82 88 76 73 87
 77 94 73 72 58 69 80 89 73 52 69 86 69 81 73 88 73 86 12 20 16  4  6 82
 69 81 73  6 16  4 71 83 82 88 73 92 88 18 77 86 38 89 77 80 88 45 82 87
 18 87 88 86 77 82 75 56 93 84 73 13  3  4  4  4  4  4  4  4  4  4  4  4
  4 80 83 91 73 86 73 72 39 83 82 87 88 86 89 71 88 83 86 18 90 69 80 89
 73 52 69 86 69 81 73 88 73 86 87  4 15 33  4 71 86 73 69 88 73 55 93 82
 88 76 73 87 77 94 73 72 58 69 80 89 73 52 69 86 69 81 73 88 73 86 12 21
 16  4  6 83 86 72 77 82 69 80  6 16  4 71 83 82 88 73 92 88 18 77 86 38
 89 77 80 88 45 82 87 18 77 82 88 56 93 84 73 13  3  4  4  4  4  4  4  4
  4  4  4  4  4 80 83 91 73 86 73 72 39 83 82 87 88 86 89 71 88 83 86 18
 90 69 80 89 73 52 69 86 69 81 73 88 73 86]


In [75]:
class Predictor:
    def __init__(self, SUFFIX_SIZE):
        self.SUFFIX_SIZE = SUFFIX_SIZE
        self.suffixToWeights = torch.ParameterDict()
        self.suffix = deque()
    
    def startNewText():
        self.suffix = deque()

    def probabilityOfNext(self, number):
        if self.suffix not in self.suffixToCounts:
            self.suffixToCounts[self.suffix] = torch.rand(ALPHABET_SIZE, dtype=torch.float)
        weights = self.suffixToCounts[self.suffix]
        probabilites = F.softmax
        total = counter.total()
        if not total:
            print('It always ended after this suffix...')
            return 0
        return counter[number] / total

    def addToSuffix(self, number):
        self.suffix.append_right(number)
        if len(suffix) > self.SUFFIX_SIZE:
            self.suffix.pop_left()
    
    def guessNext(self):
        counter = self.suffixToCounts[self.suffix]
        total = counter.total()
        if not total:
            print('No data to base guess on')
            return 0
        return counter.most_common(1)[0]
    
    
    
    

In [ ]:
def train(predictor, )